In [3]:
import pandas as pd
import xgboost as xg 
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [8]:
data = pd.read_csv('clean_cs-training.csv')

data

,gender,marital status,no_of_dependent,type of residence,educational_attainment,employment_status,current_employer,monthly net income,work_start_date,work_email_validated,...,loan_requests,failed_loan_requests,successful_loan_requests,loans,lenders,first_account,last_account,phone_numbers,created_on,status_id
0,Male,Single,1,Rented,MSc,Employed,Hancock-Bradshaw,5411,2019-07-30,True,...,1,2,7,3,19,2014-08-01,2023-07-05,2,2022-01-30,78
1,Male,Married,1,Rented,PhD,Employed,"Gutierrez, Blair and Robinson",8508,2017-07-06,True,...,8,1,2,3,2,2018-12-20,2023-09-27,1,2020-07-09,52
2,Male,Married,3,Rented,PhD,Unemployed,Ingram-Williams,1664,2015-04-02,True,...,9,2,7,0,6,2021-12-04,2023-08-10,1,2023-01-20,41
3,Male,Single,2,Rented,MSc,Unemployed,Perry Group,3350,2014-12-21,True,...,1,4,0,2,8,2017-08-03,2023-05-24,2,2020-04-17,10
4,Male,Married,1,Own House,BSc,Employed,Smith-Gardner,2951,2023-04-30,True,...,7,0,2,4,19,2021-01-03,2023-10-22,1,2021-04-28,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,Female,Married,0,Rented,PhD,Unemployed,Rodriguez and Sons,7821,2020-01-27,False,...,8,3,6,4,7,2018-11-25,2023-11-22,1,2022-06-23,64
1028,Male,Divorced,1,With Parents,BSc,Unemployed,"Wilson, Jackson and Ryan",3004,2024-02-15,False,...,6,2,3,3,3,2014-10-19,2024-01-22,1,2023-08-17,65
1029,Female,Divorced,2,Rented,MSc,Self-employed,Wilson Inc,5967,2023-11-14,False,...,0,4,0,1,6,2016-10-27,2022-07-23,2,2022-06-19,91
1030,Female,Single,4,Own House,PhD,Unemployed,Farmer-Sutton,6294,2023-05-09,True,...,0,2,6,0,13,2023-02-24,2023-08-24,2,2023-10-29,82


In [9]:
info = data.info()
summary = data.columns

print(info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    1032 non-null   object
 1   marital status            1032 non-null   object
 2   no_of_dependent           1032 non-null   int64 
 3   type of residence         1032 non-null   object
 4   educational_attainment    1032 non-null   object
 5   employment_status         1032 non-null   object
 6   current_employer          1032 non-null   object
 7   monthly net income        1032 non-null   int64 
 8   work_start_date           1032 non-null   object
 9   work_email_validated      1032 non-null   bool  
 10  country                   1032 non-null   object
 11  city                      1032 non-null   object
 12  requested_amount          1032 non-null   int64 
 13  purpose                   1032 non-null   object
 14  proposed_payday         

In [10]:
# columns_to_drop = ['sector_of_employment','org_id', 'user_id', 'loan id', 'photo url', 'work_email', 'bank', 'os_version', 'mobile_os', 'device_name', 'logins', 'LGA', 'phone_network', 'lending_lenders', 'emails', 'bank']

# drop = ['sector_of_employment']

# data = data.drop(columns=drop)
data.shape

(1032, 30)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    1032 non-null   object
 1   marital status            1032 non-null   object
 2   no_of_dependent           1032 non-null   int64 
 3   type of residence         1032 non-null   object
 4   educational_attainment    1032 non-null   object
 5   employment_status         1032 non-null   object
 6   current_employer          1032 non-null   object
 7   monthly net income        1032 non-null   int64 
 8   work_start_date           1032 non-null   object
 9   work_email_validated      1032 non-null   bool  
 10  country                   1032 non-null   object
 11  city                      1032 non-null   object
 12  requested_amount          1032 non-null   int64 
 13  purpose                   1032 non-null   object
 14  proposed_payday         

In [12]:
# Save cleaned data
# data.to_csv('clean_cs-training.csv', index=False)

In [13]:
# Process datetime data
format_list = ['%Y-%m-%d']

date_columns = ['work_start_date', 'proposed_payday', 'loan_request_day', 'first_account', 'last_account', 'created_on']

for col in data[date_columns]: 
    for fmt in format_list:
        try:
            data[col] = pd.to_datetime(data[col]).dt.year
            break 
        except ValueError:
            pass 

In [14]:
# Encode string data
# categorical_columns = ['gender', 'country', 'city', 'language', 'selfie_bvn_check', 'selfie_id_check', 'type of residence', 'marital status', 'educational_attainment', 'employment_status','sector_of_employment', 'current_employer', ]

le = LabelEncoder()

for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column])
        
data['selfie_bvn_check'] = data['selfie_bvn_check'].astype(int)
data['selfie_id_check'] = data['selfie_id_check'].astype(int)
data['work_email_validated'] = data['work_email_validated'].astype(int)


In [15]:
# Check again to see if all are in the right format
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   gender                    1032 non-null   int64
 1   marital status            1032 non-null   int64
 2   no_of_dependent           1032 non-null   int64
 3   type of residence         1032 non-null   int64
 4   educational_attainment    1032 non-null   int64
 5   employment_status         1032 non-null   int64
 6   current_employer          1032 non-null   int64
 7   monthly net income        1032 non-null   int64
 8   work_start_date           1032 non-null   int32
 9   work_email_validated      1032 non-null   int64
 10  country                   1032 non-null   int64
 11  city                      1032 non-null   int64
 12  requested_amount          1032 non-null   int64
 13  purpose                   1032 non-null   int64
 14  proposed_payday           1032 non-null 

In [16]:
# Separate  features and target label

x = data.drop('status_id', axis=1)
y = data['status_id']

In [17]:
# Initialize model and train on the data
xg_model = xg.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    max_depth=12,
    learning_rate=0.01,
    colsample_bytree=0.5
)

xg_model.fit(x, y)

/home/kelechi/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:22:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "eval_method" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_method='rmse', eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [18]:
# Check important features
booster = xg_model.get_booster()

feature_importance = booster.get_score(importance_type='gain')

feature_importance

{'gender': 52.34090042114258,
 'marital status': 65.08482360839844,
 'no_of_dependent': 90.91046142578125,
 'type of residence': 78.7470474243164,
 'educational_attainment': 86.10330200195312,
 'employment_status': 87.21261596679688,
 'current_employer': 143.7548828125,
 'monthly net income': 157.5016326904297,
 'work_start_date': 123.1712417602539,
 'work_email_validated': 107.9237060546875,
 'country': 202.4521026611328,
 'city': 200.348876953125,
 'requested_amount': 210.59812927246094,
 'purpose': 157.66380310058594,
 'loan_request_hour': 180.3955535888672,
 'loan_request_day': 153.43634033203125,
 'age': 183.27256774902344,
 'selfie_bvn_check': 164.5871124267578,
 'selfie_id_check': 157.8338623046875,
 'loan_requests': 184.450439453125,
 'failed_loan_requests': 178.2594451904297,
 'successful_loan_requests': 170.5683135986328,
 'loans': 184.64950561523438,
 'lenders': 222.9583740234375,
 'first_account': 204.12782287597656,
 'last_account': 177.77162170410156,
 'phone_numbers': 18

#### Testing/Accuracy check

In [20]:
test_data = pd.read_csv('clean_cs-test.csv')

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    32 non-null     object
 1   marital status            32 non-null     object
 2   no_of_dependent           32 non-null     int64 
 3   type of residence         32 non-null     object
 4   educational_attainment    32 non-null     object
 5   employment_status         32 non-null     object
 6   current_employer          32 non-null     object
 7   monthly net income        32 non-null     object
 8   work_start_date           32 non-null     object
 9   work_email_validated      32 non-null     bool  
 10  country                   32 non-null     object
 11  city                      32 non-null     object
 12  requested_amount          32 non-null     int64 
 13  purpose                   32 non-null     object
 14  proposed_payday           32

In [21]:

# Encode string and categorical data ffoor string
# categorical_columns = ['gender', 'country', 'city', 'selfie_bvn_check', 'selfie_id_check', 'type of residence', 'marital status', 'educational_attainment', 'employment_status','sector_of_employment', 'current_employer', ]

le = LabelEncoder()

for column in test_data.columns:
    if test_data[column].dtype == 'object':
        test_data[column] = le.fit_transform(test_data[column])
        
test_data['selfie_bvn_check'] = test_data['selfie_bvn_check'].astype(int)
test_data['selfie_id_check'] = test_data['selfie_id_check'].astype(int)
test_data['work_email_validated'] = test_data['work_email_validated'].astype(int)




In [22]:
# PROcess time/date data
format_list = ['%Y-%m-%d']

date_columns = ['work_start_date', 'proposed_payday', 'loan_request_day', 'first_account', 'last_account', 'created_on']

for col in test_data[date_columns]: 
    for fmt in format_list:
        try:
            test_data[col] = pd.to_datetime(test_data[col]).dt.year
            break 
        except ValueError:
            pass 
        
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   gender                    32 non-null     int64
 1   marital status            32 non-null     int64
 2   no_of_dependent           32 non-null     int64
 3   type of residence         32 non-null     int64
 4   educational_attainment    32 non-null     int64
 5   employment_status         32 non-null     int64
 6   current_employer          32 non-null     int64
 7   monthly net income        32 non-null     int64
 8   work_start_date           32 non-null     int32
 9   work_email_validated      32 non-null     int64
 10  country                   32 non-null     int64
 11  city                      32 non-null     int64
 12  requested_amount          32 non-null     int64
 13  purpose                   32 non-null     int64
 14  proposed_payday           32 non-null     in

In [25]:
# Check test accuracy

# x_test = test_data.drop(columns=['status_id'], axis=1)
# y_test = test_data['status_id']

# Using section of training data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,shuffle=True)


model_accuracy = 100 * xg_model.score(x_test, y_test)

print(f'The model has an accuracy of {model_accuracy:.2f}%')

The model has an accuracy of 100.00%


The lack of proper training data resulted in overfitting, And using the provided test data een gave a worse accuracy..Negative accuracy

## Using my own data(Official Kaggle data)

In [5]:
# Read from file
my_data = pd.read_csv('credit_score_data.csv')
my_data = my_data.drop(columns=['Customer_ID', 'ID', 'SSN'], axis=1)
my_data

,Month,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,1,Aaron Maashoh,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,Good,809.98,26.822620,265.0,No,49.574949,21.465380,High_spent_Small_value_payments,312.494089,Good
1,2,Aaron Maashoh,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,Good,809.98,31.944960,266.0,No,49.574949,21.465380,Low_spent_Large_value_payments,284.629162,Good
2,3,Aaron Maashoh,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,Good,809.98,28.609352,267.0,No,49.574949,21.465380,Low_spent_Medium_value_payments,331.209863,Good
3,4,Aaron Maashoh,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,Good,809.98,31.377862,268.0,No,49.574949,21.465380,Low_spent_Small_value_payments,223.451310,Good
4,5,Aaron Maashoh,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,Good,809.98,24.797347,269.0,No,49.574949,21.465380,High_spent_Medium_value_payments,341.489231,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4,Nicks,25.0,Mechanic,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,Good,502.38,34.663572,378.0,No,35.104023,24.028477,High_spent_Large_value_payments,479.866228,Poor
99996,5,Nicks,25.0,Mechanic,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,Good,502.38,40.565631,379.0,No,35.104023,24.028477,High_spent_Medium_value_payments,496.651610,Poor
99997,6,Nicks,25.0,Mechanic,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,Good,502.38,41.255522,380.0,No,35.104023,24.028477,High_spent_Large_value_payments,516.809083,Poor
99998,7,Nicks,25.0,Mechanic,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,Good,502.38,33.638208,381.0,No,35.104023,24.028477,Low_spent_Large_value_payments,319.164979,Standard


In [6]:
# encode string and categorical data 

le = LabelEncoder()
for column in my_data.columns:
    if my_data[column].dtype == 'object':
        my_data[column] = le.fit_transform(my_data[column])

In [7]:
# Check if they are in the correct format 
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  int64  
 1   Name                      100000 non-null  int64  
 2   Age                       100000 non-null  float64
 3   Occupation                100000 non-null  int64  
 4   Annual_Income             100000 non-null  float64
 5   Monthly_Inhand_Salary     100000 non-null  float64
 6   Num_Bank_Accounts         100000 non-null  float64
 7   Num_Credit_Card           100000 non-null  float64
 8   Interest_Rate             100000 non-null  float64
 9   Num_of_Loan               100000 non-null  float64
 10  Type_of_Loan              100000 non-null  int64  
 11  Delay_from_due_date       100000 non-null  float64
 12  Num_of_Delayed_Payment    100000 non-null  float64
 13  Changed_Credit_Limit      100000 non-null  fl

In [10]:
# Check Labels classes

target_encoding = my_data['Credit_Score'].unique()


print(le.classes_)
print(target_encoding)

['Good' 'Poor' 'Standard']
[0 2 1]


In [11]:
my_x = my_data.drop(columns=['Credit_Score'], axis=1)
my_y = my_data['Credit_Score']

my_x_train, my_x_test, my_y_train, my_y_test = train_test_split(my_x, my_y,test_size=0.20,shuffle=True)

my_x_train.shape

(80000, 24)

In [12]:
xg_model2 = xg.XGBClassifier(
    objective='multi:softprob',
    n_estimators=100,
    max_depth=10,
    learning_rate=0.01,
)

xg_model2.fit(my_x_train, my_y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [15]:
booster2 = xg_model2.get_booster()

feature_importance2 = booster2.get_score(importance_type='gain')

feature_importance2

{'Month': 11.876416206359863,
 'Name': 4.059664726257324,
 'Age': 3.6853079795837402,
 'Occupation': 3.779430866241455,
 'Annual_Income': 4.88038444519043,
 'Monthly_Inhand_Salary': 5.86622428894043,
 'Num_Bank_Accounts': 7.728384971618652,
 'Num_Credit_Card': 24.814050674438477,
 'Interest_Rate': 69.87323760986328,
 'Num_of_Loan': 5.611331462860107,
 'Type_of_Loan': 4.472537994384766,
 'Delay_from_due_date': 14.242058753967285,
 'Num_of_Delayed_Payment': 7.308226585388184,
 'Changed_Credit_Limit': 13.251559257507324,
 'Num_Credit_Inquiries': 5.224299907684326,
 'Credit_Mix': 1047.7987060546875,
 'Outstanding_Debt': 88.52918243408203,
 'Credit_Utilization_Ratio': 1.4425994157791138,
 'Credit_History_Age': 3.296522617340088,
 'Payment_of_Min_Amount': 1.166367769241333,
 'Total_EMI_per_month': 7.454498767852783,
 'Amount_invested_monthly': 4.649447917938232,
 'Payment_Behaviour': 3.7964236736297607,
 'Monthly_Balance': 2.3390562534332275}

In [16]:
test_score = 100 * xg_model2.score(my_x_test, my_y_test)

print(f'The model has an accuracy of {test_score:.2f}%')

The model has an accuracy of 75.02%


In [17]:
preds = xg_model2.predict(my_x_test)

print(classification_report(my_y_test, preds))

              precision    recall  f1-score   support

           0       0.61      0.72      0.66      3564
           1       0.77      0.73      0.75      5693
           2       0.80      0.77      0.78     10743

    accuracy                           0.75     20000
   macro avg       0.73      0.74      0.73     20000
weighted avg       0.76      0.75      0.75     20000



In [18]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(xg_model2, my_x_test, my_y_test, cv=5)

cv_score

array([0.72775, 0.7215 , 0.74   , 0.72825, 0.72175])

In the end, my model performed better with my own data, more than the initial dataset given to me

In [19]:
# Save the trained model
xg_model2.save_model('credit_model.json')

### Making predictions with the trained model
Short demo of the model

In [20]:
my_data

,Month,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,1,11,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,1,809.98,26.822620,265.0,1,49.574949,21.465380,2,312.494089,0
1,2,11,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,1,809.98,31.944960,266.0,1,49.574949,21.465380,3,284.629162,0
2,3,11,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,1,809.98,28.609352,267.0,1,49.574949,21.465380,4,331.209863,0
3,4,11,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,1,809.98,31.377862,268.0,1,49.574949,21.465380,5,223.451310,0
4,5,11,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,...,1,809.98,24.797347,269.0,1,49.574949,21.465380,1,341.489231,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4,6508,25.0,9,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,1,502.38,34.663572,378.0,1,35.104023,24.028477,0,479.866228,1
99996,5,6508,25.0,9,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,1,502.38,40.565631,379.0,1,35.104023,24.028477,1,496.651610,1
99997,6,6508,25.0,9,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,1,502.38,41.255522,380.0,1,35.104023,24.028477,0,516.809083,1
99998,7,6508,25.0,9,39628.99,3359.415833,4.0,6.0,7.0,2.0,...,1,502.38,33.638208,381.0,1,35.104023,24.028477,3,319.164979,2


In [33]:
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


# values = [5,	11,	23.0,	12,	19114.12,	1824.843333,	3.0,	4.0,	3.0,	4.0	...	1	809.98	24.797347	269.0	1	49.574949	21.465380	1	341.489231	0] 
vals = [[8,	11	,23,	12,	19114.12,	1824.8433333333,	3,	4,	3,	4,	136,	3,	6,	11.27,	4,	1,	809.98,	23.933794802,	272,	1,	49.5749492149,	21.4653802647,	1,	358.1241676094]]
# instance = np.array(vals)
# vals
# instance

print(my_data.columns)

Index(['Month', 'Name', 'Age', 'Occupation', 'Annual_Income',
       'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')


In [51]:
vals = [my_data.loc[143]]
print(vals)

[Month                           8.000000
Name                         6886.000000
Age                            46.000000
Occupation                     13.000000
Annual_Income               32284.620000
Monthly_Inhand_Salary        2898.385000
Num_Bank_Accounts               6.000000
Num_Credit_Card                 7.000000
Interest_Rate                  17.000000
Num_of_Loan                     6.000000
Type_of_Loan                 1128.000000
Delay_from_due_date            13.000000
Num_of_Delayed_Payment         12.000000
Changed_Credit_Limit            2.220000
Num_Credit_Inquiries           10.000000
Credit_Mix                      2.000000
Outstanding_Debt             1283.370000
Credit_Utilization_Ratio       27.930124
Credit_History_Age             84.000000
Payment_of_Min_Amount           2.000000
Total_EMI_per_month           103.037560
Amount_invested_monthly        35.793568
Payment_Behaviour               5.000000
Monthly_Balance               117.029107
Credit_Score   

In [53]:
# Smaple predictions and data

classes = ['Good', 'Poor', 'Standard']

def predict(sample_data):
    predictions = xg_model2.predict_proba(sample_data)
    pred_class = np.argmax(predictions)
    certainty = 100 * np.max(predictions)
    
    return pred_class, certainty

standard_credit_vals = [[8,	6886, 46,	13,	32284.62,	2898.385,	6,	7,	17,	6,	1128,	13,	12,	2.22,	10,	2,	1283.37,	27.9301237106,	84,	2,	103.0375603554,	35.7935684142,	5,	117.0291073593]]

standard_cs_score_predictions = predict(standard_credit_vals)

pred_class, certainty = standard_cs_score_predictions

print(f"Class: {classes[pred_class]}")
print(f"{certainty:.2f}%")

Class: Standard
60.76%


In [55]:
good_credit_vals = [[8,	11	,23, 12, 19114.12,
         1824.8433333333, 3,	4,	3,	4,	136, 3,	6,	11.27,
         4,	1,	809.98,	23.933794802, 272,	1, 49.5749492149,	
         21.4653802647,	1,	358.1241676094]]

good_cs_score_prediction = predict(good_credit_vals)

pred_class, certainty = good_cs_score_prediction

print(f"Class: {classes[pred_class]}")
print(f"{certainty:.2f}%")


Class: Good
43.63%


In [56]:
bad_credit_vals = [[1,	7494,	34,	9,	57983.12,	4720.9266666667,	9,	8,	31,	8,	4573,	47,	19,	7.27,	12,	0,	3532.83,	32.3217694616,	147,	2,	200.4326133947,	159.1367219501,	5,	295.9538413234]]

bad_cs_score_prediction = predict(bad_credit_vals)

pred_class, certainty = bad_cs_score_prediction

print(f"Class: {classes[pred_class]}")
print(f"{certainty:.2f}%")

Class: Poor
64.31%
